# Import Necessary Libraries

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train_new.csv')
train = train.sample(n=10000).reset_index(drop=True)
train.head()

,image,class
0,v_BlowDryHair_g16_c04.avi_frame2.jpg,BlowDryHair
1,v_JavelinThrow_g16_c04.avi_frame3.jpg,JavelinThrow
2,v_BasketballDunk_g21_c03.avi_frame0.jpg,BasketballDunk
3,v_BoxingSpeedBag_g11_c03.avi_frame0.jpg,BoxingSpeedBag
4,v_JumpingJack_g18_c01.avi_frame0.jpg,JumpingJack


In [ ]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

In [4]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [5]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

Using a different base model to evalute if this model architecture leads to better performance

In [6]:
# creating the base model of pre-trained VGG16 model
base_model = VGG19(weights='imagenet', include_top=False)

In [7]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(8000, 7, 7, 512)

In [9]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(2000, 7, 7, 512)

In [10]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(8000, 7*7*512)
X_test = X_test.reshape(2000, 7*7*512)

In [11]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [12]:
# shape of images
X_train.shape

(8000, 25088)

In [13]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

In [14]:
# defining a function to save the weights of best model
from tensorflow.keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('vgg19weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [15]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [16]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
63/63 [==============================] - 8s 125ms/step - loss: 4.6350 - accuracy: 0.0135 - val_loss: 4.5812 - val_accuracy: 0.0380
Epoch 2/200
63/63 [==============================] - 7s 104ms/step - loss: 4.5623 - accuracy: 0.0192 - val_loss: 4.5240 - val_accuracy: 0.0410
Epoch 3/200
63/63 [==============================] - 6s 100ms/step - loss: 4.4538 - accuracy: 0.0362 - val_loss: 4.2753 - val_accuracy: 0.0690
Epoch 4/200
63/63 [==============================] - 6s 99ms/step - loss: 4.2126 - accuracy: 0.0640 - val_loss: 3.9525 - val_accuracy: 0.0835
Epoch 5/200
63/63 [==============================] - 6s 100ms/step - loss: 3.9785 - accuracy: 0.0869 - val_loss: 3.6941 - val_accuracy: 0.1420
Epoch 6/200
63/63 [==============================] - 6s 101ms/step - loss: 3.7918 - accuracy: 0.1154 - val_loss: 3.4739 - val_accuracy: 0.1925
Epoch 7/200
63/63 [==============================] - 6s 101ms/step - loss: 3.6323 - accuracy: 0.1390 - val_loss: 3.3651 - val_accuracy: 0.2135


63/63 [==============================] - 6s 92ms/step - loss: 1.5260 - accuracy: 0.5576 - val_loss: 2.0976 - val_accuracy: 0.4615
Epoch 116/200
63/63 [==============================] - 6s 93ms/step - loss: 1.5622 - accuracy: 0.5573 - val_loss: 2.1417 - val_accuracy: 0.4515
Epoch 117/200
63/63 [==============================] - 6s 92ms/step - loss: 1.5597 - accuracy: 0.5475 - val_loss: 2.1431 - val_accuracy: 0.4280
Epoch 118/200
63/63 [==============================] - 6s 92ms/step - loss: 1.5484 - accuracy: 0.5546 - val_loss: 2.1689 - val_accuracy: 0.4315
Epoch 119/200
63/63 [==============================] - 6s 92ms/step - loss: 1.5442 - accuracy: 0.5589 - val_loss: 2.0597 - val_accuracy: 0.4460
Epoch 120/200
63/63 [==============================] - 6s 93ms/step - loss: 1.5260 - accuracy: 0.5633 - val_loss: 2.1455 - val_accuracy: 0.4550
Epoch 121/200
63/63 [==============================] - 6s 92ms/step - loss: 1.6174 - accuracy: 0.5389 - val_loss: 2.1705 - val_accuracy: 0.4435
Epoch 

Epoch 172/200
63/63 [==============================] - 6s 93ms/step - loss: 1.3785 - accuracy: 0.6019 - val_loss: 2.2709 - val_accuracy: 0.4455
Epoch 173/200
63/63 [==============================] - 6s 92ms/step - loss: 1.3947 - accuracy: 0.6030 - val_loss: 2.4384 - val_accuracy: 0.4300
Epoch 174/200
63/63 [==============================] - 6s 93ms/step - loss: 1.4476 - accuracy: 0.5869 - val_loss: 2.3063 - val_accuracy: 0.4610
Epoch 175/200
63/63 [==============================] - 6s 93ms/step - loss: 1.4091 - accuracy: 0.5956 - val_loss: 2.3447 - val_accuracy: 0.4525
Epoch 176/200
63/63 [==============================] - 6s 93ms/step - loss: 1.4188 - accuracy: 0.5900 - val_loss: 2.3204 - val_accuracy: 0.4250
Epoch 177/200
63/63 [==============================] - 6s 94ms/step - loss: 1.3970 - accuracy: 0.5982 - val_loss: 2.4120 - val_accuracy: 0.4370
Epoch 178/200
63/63 [==============================] - 6s 93ms/step - loss: 1.3989 - accuracy: 0.5999 - val_loss: 2.3141 - val_accuracy:

In [17]:
# loading the trained weights
model.load_weights("vgg19weight.hdf5")

In [18]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [19]:
# getting the test list
f = open("ucfTrainTestlist/testlist02.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi


In [22]:
# creating the tags
train = pd.read_csv('train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [24]:
import cv2
import math
import os
from glob import glob
from scipy import stats as s

In [25]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('UCF-101/'+videoFile.split(' ')[0])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])

  0%|                                                                                         | 0/3734 [00:00<?, ?it/s]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


100%|██████████████████████████████████████████████████████████████████████████████| 3734/3734 [42:17<00:00,  1.47it/s]


In [26]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

29.619710765934652

# Model Evaluation

Similar to VGG16, the updated VGG19 architecture only led to minor improvement even though the amount of data is the same. More models should be iterated before decided on a good base model to further investigation